### https://tschaume.github.io/usta_tennis/usta_teams.html

Source code for scraping the data is [here](https://github.com/tschaume/usta_tennis). 27,377 USTA NorCal Pages as of 04/25/2020 (~120 pages/min).

In [ ]:
from pandas import DataFrame
from pymongo import MongoClient
client = MongoClient()
db = client['usta']

In [ ]:
print("teams:", db.teams.count_documents({}))
print("registrations:", db.registrations.count_documents({}))

In [ ]:
# for y in ["2016"]:
#     print(len(db.teams.distinct(
#         "league_url",
#         {"season_name": f"{y} Mixed 40 & Over", "gender": {"$in": ["M", "W"]}},
#     )))
db.teams.find_one({"id": 84866})

In [ ]:
db.registrations.find_one({"id": '53045_54164'})

## Overview

In [ ]:
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "notebook"

In [ ]:
from collections import Counter, defaultdict

def get_season(comp, age):
    return f"{comp} {age} & Over"


def get_figure(season_groups, min_matches=0):
    print(f"min_matches: {min_matches}")
    data = defaultdict(list)
    hdata = defaultdict(list)
    years = list(map(str, db.teams.distinct("year")))
    years.remove("2012")
    years.remove("2014")
    years.remove("2020")

    for year in years:

        container = defaultdict(dict)
        for group, seasons in season_groups.items():
            print(year, group)

            teams = []
            for season in seasons:
                q = {
                    "year": year,
                    "season_name": f"{year} {season}",
                    "gender": {"$in": ["M", "W", "O"]}  # exclude daytime
                    #"level": "3.0"
                }

                pipeline = [
                    {"$match": {"team_id": {"$in": db.teams.distinct("id", q)}}},
#                     {"$project": {
#                         "team_id": 1, "player_id": 1, "id": 1, "win": 1, "loss": 1
#                     }}
                ]

                if group.endswith("Only"):
                    ref_group = group.replace(" Only", "")
                    if "18" in ref_group:
                        ref_group = ref_group.replace("18", "40")
                    elif "40" in ref_group:
                        ref_group = ref_group.replace("40", "18")
                    pipeline.append({"$match": {
                        "player_id": {"$nin": list(container[ref_group]["players"])}
                    }})


                pipeline.append({"$group": {
                    "_id": "$team_id", "registrations": {
                        "$push": {
                            "id": "$id", "pid": "$player_id",
                            "matches": {"$add": ["$win", "$loss"]},
                            "postseason": {"$add": ["$playoffs", "$districts", "$sectionals"]}
                        }
                    }
                }})

                teams += list(db.registrations.aggregate(pipeline))


            container[group] = defaultdict(set)
            for t in teams:
                spots = 0
                for r in t["registrations"]:
                    nmatches = r["matches"] - r["postseason"]
                    spots += nmatches
                    if nmatches >= min_matches:
                        container[group]["registrations"].add(r["id"])
                        container[group]["players"].add(r["pid"])  
                        
                hdata['year'].append(int(year))
                hdata['group'].append(group)
                hdata['category'].append('nregs_per_spot')
                hdata['number'].append(spots / len(t["registrations"]))  
                hdata['id'].append(t["_id"])

            for k, v in container[group].items():
                data['year'].append(int(year))
                data['group'].append(group)
                data['category'].append(k)
                data['number'].append(len(v))

    #df = DataFrame(data=data)
    #print(df)
    #fig = px.scatter(df, x='year', y='number', facet_col='category', color="group", trendline="ols")
    #fig.show()
    #return fig
    
    return DataFrame(data=hdata)
    
    
    

def calculate_fees(year, season_groups, min_matches=0):
    fig = get_figure(season_groups, min_matches=min_matches)
    results = px.get_trendline_results(fig)
    
    fees = defaultdict(dict)
    league_fees = {
        #2011: 23, 2012: 25,
        2013: 26, 2014: 26, 2015: 27,
        2016: 28, 2017: 28, 2019: 28,
        2020: 29
    }
    facility_fee = 25
    membership_fee = 44

    for group in season_groups.keys():
        params = {
            cat: results.query(
                f"group == '{group}' and category == '{cat}'"
            ).px_fit_results.iloc[0].params
            for cat in ["registrations", "players"]
        }

        registrations = params["registrations"][0] + params["registrations"][1] * year
        players = params["players"][0] + params["players"][1] * year
        #print(year, registrations, params)
        fees[group]["league"] = registrations * league_fees[year]
        fees[group]["facility"] = registrations * facility_fee if year > 2019 else 0
#         fees[group]["league"] = players * league_fees[year]
#         fees[group]["facility"] = players * facility_fee if year > 2019 else 0
        fees[group]["membership"] = players * membership_fee

    return fees

### Trend and projections

In [ ]:
season_groups = {}
for comp in ["Adult"]:#, "Mixed"]:
    #season_groups[comp] = get_seasons(comp, [18, 40])
    season_groups[f"{comp} 40 & Over"] = [get_season(comp, 40)]
    season_groups[f"{comp} 18 & Over"] = [get_season(comp, 18)]
    #season_groups[f"{comp} 40 & Over Only"] = get_seasons(comp, [40])
    #season_groups[f"{comp} 18 & Over Only"] = get_seasons(comp, [18])
    #season_groups[f"{comp} 40/55/65/70+"] = get_seasons(comp, [40, 55, 65, 70])

fees = calculate_fees(2020, season_groups)

In [ ]:
print(fees)
total_fees = Counter()
for dct in fees.values():
    for k, v in dct.items():
        total_fees[k] += v
        
total_fees    

### Team Viability

In [ ]:
name = "Adult & Mixed"
season_groups = {
    name: [
        get_season(comp, age)
        for age in ["18", "40", "55", "65", "70"]
        for comp in ["Adult", "Mixed"]
    ]
}

In [ ]:
df = get_figure(season_groups)

In [ ]:
px.histogram(df, x="number", nbins=40) #marginal="rug", hover_data=df.columns)


### Revenue

In [ ]:

revenues = {}
revenues["2019"] = calculate_fees(2019, season_groups)[name]

for min_matches in [0, 3, 4]:
    revenues[str(min_matches)] = calculate_fees(2020, season_groups, min_matches=min_matches)[name]
    
revenues

In [ ]:
data = defaultdict(list)
xaxis = "expected minimum number of matches (N) per registration with facility fee in effect"
yaxis = "difference in revenue vs 2019"
color = "fee type"

for k in list(revenues.keys())[1:]:
    scenario = "no change" if k == "0" else f"N = {k}"
    for typ, revenue in revenues[k].items():
        data[xaxis].append(scenario)
        data[color].append(typ)
        data[yaxis].append(revenue - revenues["2019"][typ])

df = DataFrame(data=data)
fig = px.bar(df, x=xaxis, y=yaxis, color=color, barmode="group")
r2019 = sum(revenues["2019"].values()) / 1e6
fig["layout"]["title"] = f"Projections for 2020 Revenue compared to ${r2019:.3g}M in 2019"
fig.show()

In [ ]:
px.data.tips()

- 2017 -> 2018 Adult 40+ change from 3 to 5 lines
  * normalize number of teams on number of registrations per team to account for roster size
    (regulations, availability and other captain decisions)
  * "
  * enable grouping arbitrary leagues and levels
  * also reveal effects of players deciding to play on less teams due to fee hike
    (level and/or area)

- 2014 -> 2015 Start of Daytime leagues

---

- players aged 40 and older, can decide one of the following every year:
  * play both "18 & Over" and "40 & Over" (this should be the majority)
  * only play "40 & Over" (this should be a small number)
  * only play "18 & Over" (this should be a negligible number)

- 18+ consists of age brackets 18-40 and 40+
- anyone who doesn't play "40 & Over" is considered younger than 40 (i.e. in the 18-40 age bracket above)
- so the assumption is that anyone who can play "40 & Over" actually does
- 


---

- 861 captains registered 936 teams for 212 organizations in the 2020 Adult 18 & Over season
- the number of 18+ captains declined by 24% over the last 8 years (14% for 40+) for both Adult and Mixed
- 2020 Adult 18+ has 281 teams less than 2013. With about 15 players per team, USTA NorCal misses out on about $120,000 in registration fees on that league alone.
- the number of organizations went down by 11\% and 6\%, respectively, over the same time period

## save seasons and teams as spreadsheets

[spreadsheet](https://docs.google.com/spreadsheets/d/1yWcF0jrou2mkADM2q9PEhWy5uLwSjZiUXX7a6LJD1uw/edit?usp=sharing) with 66 seasons and 26822 teams in total

- each tab contains all teams for one season
- daytime, combo or tri-level not included
- columns: team, year, gender, level, captain, city, area, organization
- direct links to the pages for teams, captains and organizations also included

In [ ]:
from pandas import ExcelWriter

def save_xls(dct, xls_path='usta_teams.xlsx'):
    writer = ExcelWriter(xls_path)
    for name, s in dct.items():
        s.to_excel(writer, sheet_name=name)
    writer.save()

In [ ]:
seasons = entries.distinct("season_name")
len(seasons)

In [ ]:
skip = ['_id', 'id', 'league', 'season_id', 'season_name']
projection = {k: 0 for k in skip}
columns = list(entries.find_one({}, projection).keys())
columns

In [ ]:
sheets = {}

for season in reversed(seasons):
    docs = entries.find({"season_name": season}, projection)
    print(season, docs.count())
    data = {c: [] for c in columns} 
    for doc in docs:
        for k, v in doc.items():
            data[k].append(v)

    season = season.replace('/', ' ')[:32]
    sheets[season] = DataFrame(data=data)
    sheets[season].set_index('team', inplace=True)
    sheets[season].sort_values(by=['year', 'gender', 'level', 'team'], inplace=True)

save_xls(sheets)
print('DONE')